# Regularization Techniques: Complete Explanation
## L2 (Ridge/Weight Decay), L1 (Lasso), and Dropout
### (Detailed Step-by-Step with Cat vs Dog Classification)

---

## 🔗 **Connection to Previous Topics**

### **What We Know So Far:**

**From Neural Networks:**
```
Training process:
1. Forward pass: Make predictions
2. Calculate loss: How wrong we are
3. Backward pass: Compute gradients
4. Update weights: w := w - α·∂L/∂w
```

**From CNNs:**
```
Cat vs Dog classifier:
Input (64×64×3 image) → Conv layers → Dense layers → Output [P(cat), P(dog)]

Network might have:
- 50,000 weights in conv layers
- 100,000 weights in dense layers
- Total: 150,000 parameters!
```

**The New Problem: OVERFITTING**

---

# Part 1: Understanding Overfitting

## 1. What is Overfitting?

### **Simple Analogy**

Imagine a student preparing for an exam:

**Good Student (Generalization):**
```
Studies: Understands concepts
Exam: Solves new problems correctly
✓ Can apply knowledge to unseen questions
```

**Overfit Student (Memorization):**
```
Studies: Memorizes every practice problem exactly
Exam: Fails on slightly different questions
✗ Only knows exact problems, can't generalize
```

**Neural networks can make the same mistake!**

---

## 2. Overfitting in Cat vs Dog Classification

### **Our Dataset:**

```
Training Set: 100 images
- 50 cats
- 50 dogs

Test Set: 20 images (never seen before)
- 10 cats  
- 10 dogs
```

### **Scenario 1: Healthy Model (Good Generalization)**

```
Training Accuracy: 95%
Test Accuracy: 93%

The model learned general features:
- "Pointy ears" → Cat
- "Floppy ears" → Dog
- "Whiskers" → Cat
- "Long snout" → Dog

✓ Performs well on new images!
```

### **Scenario 2: Overfit Model (Memorization)**

```
Training Accuracy: 100%
Test Accuracy: 65%

The model memorized specific images:
- "This exact pixel pattern at position (23,45)" → Cat
- "This specific noise pattern" → Dog
- "Training image #37's exact colors" → Cat

✗ Fails on new images! Too specific!
```

**Visualize the problem:**

```
         UNDERFITTING          GOOD FIT           OVERFITTING
         
Train:   ●  ●  ○  ○          ●  ●  ○  ○          ●  ●  ○  ○
         ●  ●  ○  ○          ●  ●  ○  ○          ●  ●  ○  ○
         
Learned: ─────────          ──────────          ╱╲╱╲╱╲╱╲╱╲
         (too simple)       (just right)        (too complex)
         
Test:    ●  ?  ○  ?         ●  ●  ○  ○          ●  ?  ○  ?
         Poor on both       Good on both!       Good train,
         train & test       85-95%              bad test!
         <70%                                   Train: 100%
                                                Test: 60%
```

---

## 3. Real Numbers: Watching Overfitting Happen

Let's train a Cat vs Dog classifier and watch it overfit!

### **Network Architecture:**

```
Input: 64×64×3 = 12,288 pixels
↓
Hidden Layer 1: 1000 neurons (12,288,000 weights!)
↓
Hidden Layer 2: 500 neurons (500,000 weights)
↓
Output Layer: 2 neurons (1,000 weights)

Total: ~12.8 MILLION parameters
Training samples: Only 100 images!

Ratio: 128,000 parameters per training sample!
This is a recipe for overfitting! 🚨
```

### **Training Progress (Epoch by Epoch):**

| Epoch | Training Loss | Training Acc | Test Loss | Test Acc | What's Happening |
|-------|--------------|--------------|-----------|----------|------------------|
| 1 | 0.693 | 50% | 0.695 | 48% | Random guessing |
| 5 | 0.420 | 78% | 0.435 | 76% | Learning general features |
| 10 | 0.210 | 92% | 0.235 | 89% | Good generalization! |
| 20 | 0.085 | 98% | 0.315 | 85% | Starting to overfit... |
| 30 | 0.025 | 100% | 0.520 | 78% | Overfitting badly! |
| 50 | 0.005 | 100% | 0.890 | 65% | Memorized training set |
| 100 | 0.001 | 100% | 1.450 | 62% | Complete overfitting |

**The Warning Signs:**

```
    Loss
     ↑
  1.5│              ╱─ Test loss rising
  1.0│            ╱  (BAD SIGN!)
  0.5│──────────╱
  0.0│───────────── Training loss falling
     └──────────────────────→ Epochs
         ↑ 
    Best point! (Epoch 10)
    Should stop here!
```

**What the weights look like:**

```
Epoch 10 (Good):
Weights: Small, smooth
w = [0.23, -0.15, 0.08, -0.31, 0.19, ...]
Most weights: -1 to +1

Epoch 100 (Overfit):
Weights: Large, chaotic
w = [15.3, -23.7, 8.9, -45.2, 31.8, ...]
Many weights: -50 to +50!

The network became too sensitive!
Tiny changes in input → huge changes in output
```

---

## 4. Why Does Overfitting Happen?

### **Reason 1: Too Many Parameters**

```
100 training images
12,800,000 parameters

It's like trying to fit 100 data points with a 
12-million-degree polynomial! You can fit perfectly
but it's meaningless.

Example with simple data:

3 points:  (1,2), (2,4), (3,5)

Fit with line (2 parameters): y = 1.5x + 0.5
  → Smooth, generalizes well ✓
  
Fit with degree-10 polynomial (11 parameters):
  → Wiggly, passes through exactly but crazy between points ✗
```

### **Reason 2: Not Enough Data**

```
Network capacity: 12M parameters
Training samples: 100

The network has too much "freedom"
Many different weight configurations
can all perfectly fit 100 images!

Like having 100 equations with 12 million unknowns
→ Infinite solutions!
```

### **Reason 3: Training Too Long**

```
Early training: Learning general patterns
Later training: Memorizing noise and specifics

It's like studying:
- First hour: Understand concepts (good!)
- Hour 10: Start memorizing exact wording
- Hour 100: Memorized every comma, can't adapt
```

---

# Part 2: L2 Regularization (Ridge / Weight Decay)

## 1. Plain English Explanation

### **The Core Idea**

**L2 regularization says:** "Keep weights small!"

**Why?** Small weights = simpler model = better generalization

```
Without L2:                    With L2:
Weights can grow huge          Penalty for large weights
w = [50, -80, 120, -200]      w = [0.5, -0.8, 1.2, -2.0]
     ↑                              ↑
Over-sensitive!                Reasonable!

Small input change → HUGE output  Small input change → Small output
Overfits to training noise        Generalizes to new data
```

### **The Intuition: Financial Penalty**

Think of it like a tax on large weights:

```
Original loss: "How wrong are predictions?"
L2 loss: "How wrong are predictions? + Penalty for large weights"

Network must balance:
- Fitting training data (low prediction error)
- Keeping weights small (low weight penalty)
```

---

## 2. The Mathematics

### **Original Loss Function:**

$$L = \frac{1}{m}\sum_{i=1}^{m}(y_i - \hat{y}_i)^2$$

Just measures prediction errors.

### **L2 Regularized Loss:**

$$L_{L2} = \frac{1}{m}\sum_{i=1}^{m}(y_i - \hat{y}_i)^2 + \frac{\lambda}{2}\sum_{j=1}^{n}w_j^2$$

**Components:**

| Part | Name | Meaning |
|------|------|---------|
| $\frac{1}{m}\sum(y_i - \hat{y}_i)^2$ | Prediction loss | How wrong predictions are |
| $\frac{\lambda}{2}\sum w_j^2$ | L2 penalty | Sum of squared weights |
| $\lambda$ | Regularization strength | How much to penalize (0.001 to 0.1 typical) |
| $w_j$ | Weight j | A specific parameter in the network |
| $n$ | Number of weights | Total parameters |

**Key insight:** Squaring weights means:
- Large weights get HUGE penalty (10² = 100)
- Small weights get tiny penalty (0.1² = 0.01)
- Network prefers many small weights over few large ones

---

## 3. Step-by-Step Numerical Example

### **Scenario: Single Neuron**

```
Input: x = [2.0, 3.0]
Weights: w = [w₁, w₂]
Bias: b = 0.5
True label: y = 1.0

Forward pass:
z = w₁×2.0 + w₂×3.0 + 0.5
ŷ = sigmoid(z)
```

### **Training Step 1: No Regularization (λ = 0)**

**Current weights:**
```
w₁ = 5.0
w₂ = 8.0
b = 0.5
```

**Forward pass:**
```
z = 5.0×2.0 + 8.0×3.0 + 0.5
  = 10.0 + 24.0 + 0.5
  = 34.5

ŷ = sigmoid(34.5) = 1/(1 + e^(-34.5)) ≈ 0.99999999

Prediction loss:
L_pred = (1.0 - 0.99999999)² = 0.00000001

Total loss (no regularization):
L = 0.00000001 ✓ (seems perfect!)
```

**Gradient (no regularization):**
```
∂L/∂w₁ = (ŷ - y) × x₁
       = (0.99999999 - 1.0) × 2.0
       = -0.00000002

∂L/∂w₂ = (ŷ - y) × x₂
       = (0.99999999 - 1.0) × 3.0
       = -0.00000003
```

**Update (α = 0.1):**
```
w₁ := 5.0 - 0.1×(-0.00000002) = 5.00000000002
w₂ := 8.0 - 0.1×(-0.00000003) = 8.00000000003

Weights barely change! They're stuck at large values!
Network is overconfident and overfit.
```

---

### **Training Step 2: With L2 Regularization (λ = 0.01)**

**Same starting weights:**
```
w₁ = 5.0
w₂ = 8.0
```

**Forward pass (same as before):**
```
ŷ = 0.99999999

Prediction loss:
L_pred = (1.0 - 0.99999999)² = 0.00000001
```

**L2 penalty:**
```
L2_penalty = (λ/2) × (w₁² + w₂²)
           = (0.01/2) × (5.0² + 8.0²)
           = 0.005 × (25 + 64)
           = 0.005 × 89
           = 0.445

Total loss:
L = L_pred + L2_penalty
  = 0.00000001 + 0.445
  = 0.445

Loss dominated by regularization!
Network says: "Your weights are too large!"
```

**Gradient with L2:**
```
∂L/∂w₁ = ∂L_pred/∂w₁ + ∂L2_penalty/∂w₁
       = -0.00000002 + λ×w₁
       = -0.00000002 + 0.01×5.0
       = -0.00000002 + 0.05
       = 0.05 (positive! wants to decrease w₁)

∂L/∂w₂ = ∂L_pred/∂w₂ + ∂L2_penalty/∂w₂
       = -0.00000003 + λ×w₂
       = -0.00000003 + 0.01×8.0
       = -0.00000003 + 0.08
       = 0.08 (positive! wants to decrease w₂)
```

**Update (α = 0.1):**
```
w₁ := 5.0 - 0.1×0.05 = 5.0 - 0.005 = 4.995
w₂ := 8.0 - 0.1×0.08 = 8.0 - 0.008 = 7.992

Weights are shrinking!
This is called "weight decay"
```

**After 100 iterations:**
```
Without L2: w = [5.00, 8.00] (stuck, overfit)
With L2:    w = [1.23, 1.98] (smaller, generalized)

The L2 penalty drove weights down!
```

---

## 4. Complete Example: Cat vs Dog Network

### **Network Architecture:**

```
Input: 12,288 features (64×64×3 image)
Hidden: 100 neurons
Output: 2 neurons (cat, dog)

Total weights: 12,288×100 + 100×2 = 1,229,000 weights!
```

### **Training: 3 Different λ Values**

**Dataset:**
- 100 training images (50 cats, 50 dogs)
- 20 test images (10 cats, 10 dogs)

---

#### **Case 1: No Regularization (λ = 0)**

**Epoch 1:**
```
Weights (sample of 5):
w = [0.02, -0.01, 0.03, -0.02, 0.01]

Training loss: 0.693
Test loss: 0.701

Weight sum: Σw² = 0.0007 (small)
```

**Epoch 50:**
```
Weights (sample of 5):
w = [45.2, -67.3, 89.1, -123.5, 156.8]

Training loss: 0.001 (perfect fit!)
Test loss: 1.850 (terrible!)

Weight sum: Σw² = 45,892 (HUGE!)

Network memorized training set!
```

**Weight evolution:**
```
    |Weight|
      ↑
  150│                     ╱
  100│                  ╱
   50│              ╱
    0│──────────╱
      └────────────────→ Epochs
      
Weights exploded!
```

---

#### **Case 2: Moderate Regularization (λ = 0.01)**

**Epoch 1:**
```
Weights (sample of 5):
w = [0.02, -0.01, 0.03, -0.02, 0.01]

L_pred = 0.693
L2_penalty = 0.01/2 × 0.0007 = 0.0000035
L_total = 0.693

Negligible penalty at start (weights small)
```

**Epoch 50:**
```
Weights (sample of 5):
w = [2.3, -1.8, 3.1, -2.7, 1.9]

Training loss: 0.085 (good fit)
Test loss: 0.095 (generalizes well!)

Weight sum: Σw² = 85.2
L2_penalty = 0.01/2 × 85.2 = 0.426

Network balanced fitting vs weight size!
```

**Weight evolution:**
```
    |Weight|
      ↑
    5│      ────────────
    0│────╱
      └────────────────→ Epochs
      
Weights grew but stabilized!
```

---

#### **Case 3: Too Strong Regularization (λ = 1.0)**

**Epoch 50:**
```
Weights (sample of 5):
w = [0.05, -0.03, 0.08, -0.06, 0.04]

Training loss: 0.520 (underfit!)
Test loss: 0.535 (consistent but poor)

Weight sum: Σw² = 0.14
L2_penalty = 1.0/2 × 0.14 = 0.07

L2 penalty dominated!
Weights stayed too small.
Network couldn't fit even training data!
```

**Weight evolution:**
```
    |Weight|
      ↑
 0.10│─────────────────
 0.05│╱
 0.00│
      └────────────────→ Epochs
      
Weights barely grew!
```

---

### **Comparison Table:**

| λ | Training Acc | Test Acc | Weight Magnitude | Status |
|---|-------------|----------|------------------|--------|
| **0** | 100% | 65% | Very large (45-150) | Overfit |
| **0.001** | 98% | 88% | Large (5-15) | Slight overfit |
| **0.01** | 94% | 92% | Medium (1-5) | **Good!** ✓ |
| **0.1** | 85% | 84% | Small (0.3-2) | Slight underfit |
| **1.0** | 68% | 67% | Very small (0.01-0.1) | Underfit |

**The sweet spot: λ = 0.01**

---

## 5. How Gradient Descent Changes with L2

### **Without L2:**

```
∂L/∂w = ∂L_pred/∂w

Update:
w := w - α × ∂L_pred/∂w

Only cares about prediction error!
```

### **With L2:**

```
∂L/∂w = ∂L_pred/∂w + λw

Update:
w := w - α × (∂L_pred/∂w + λw)
  = w - α×∂L_pred/∂w - α×λw
  = w(1 - αλ) - α×∂L_pred/∂w
      ↑           ↑
   "decay"    usual gradient

Weight decay factor: (1 - αλ)
```

**Numerical example:**
```
w = 10.0
α = 0.1
λ = 0.01
∂L_pred/∂w = 2.0

Without L2:
w := 10.0 - 0.1×2.0 = 10.0 - 0.2 = 9.8

With L2:
w := 10.0 - 0.1×2.0 - 0.1×0.01×10.0
  = 10.0 - 0.2 - 0.01
  = 9.79

Extra 0.01 from weight decay!
```

**Over many iterations:**
```
Iteration 0:   w = 10.00
Iteration 1:   w = 9.79
Iteration 10:  w = 8.52
Iteration 100: w = 3.21
Iteration 500: w = 1.15

Weight gradually shrinks!
```

---

## 6. Visualizing L2's Effect

### **Loss Landscape Without L2:**

```
         w₂
          ↑
        5 │     ╱╲╱╲╱╲
          │   ╱        ╲
        0 │─╱   (min)   ╲─→ w₁
          │              
       -5 │
       
Network can reach minimum
with huge weights (overfit)
Many equivalent solutions
```

### **Loss Landscape With L2:**

```
         w₂
          ↑
        5 │
          │     ╱─╲
        0 │───●─────→ w₁
          │   min
       -5 │
       
L2 adds a "bowl" penalty
centered at origin
Network prefers solutions
near (0,0) - small weights!
```

**Combined landscape:**

```
         Loss
          ↑
          │      ┌─ L2 penalty (bowl shape)
          │     ╱│╲
          │   ╱  │  ╲   ← Combined loss
          │ ╱   ●│    ╲  (shifted minimum)
          │╱_____│_____╲
          └──────┴───────→ Weight magnitude
                 ↑
            New minimum
         (smaller weights)
```

---

## 7. Why L2 Prevents Overfitting

### **Reason 1: Limits Model Complexity**

```
Without L2:
Any weight configuration allowed
→ Can memorize training data

With L2:
Large weights heavily penalized
→ Forces simpler model
→ Simpler model = better generalization
```

### **Reason 2: Reduces Sensitivity**

```
Output = w₁x₁ + w₂x₂ + ... + wₙxₙ

Large weights:
If w₁ = 100, small noise in x₁ (±0.01)
causes huge output change (±1.0)
→ Overly sensitive to noise!

Small weights:
If w₁ = 1.0, noise in x₁ (±0.01)
causes small output change (±0.01)
→ Robust to noise!
```

### **Reason 3: Spreads Information**

```
Without L2:
Model might use: w = [100, 0, 0, 0, 0]
Relies on single feature (brittle!)

With L2:
Model forced to use: w = [0.5, 0.4, 0.3, 0.6, 0.5]
Uses many features (robust!)

If one feature fails → Others compensate
```

---

## 8. Practical Implementation

### **PyTorch-style Code:**

```python
# Without L2
loss = criterion(outputs, targets)
optimizer.zero_grad()
loss.backward()
optimizer.step()

# With L2 (Method 1: Manual)
lambda_l2 = 0.01
l2_penalty = 0
for param in model.parameters():
    l2_penalty += torch.sum(param ** 2)

loss = criterion(outputs, targets) + (lambda_l2 / 2) * l2_penalty
optimizer.zero_grad()
loss.backward()
optimizer.step()

# With L2 (Method 2: Built-in weight_decay)
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.1,
    weight_decay=0.01  # This is λ!
)

loss = criterion(outputs, targets)
optimizer.zero_grad()
loss.backward()
optimizer.step()  # Automatically applies weight decay
```

### **Step-by-step: One training batch**

```python
# Forward pass
batch_x = [cat_image_1, dog_image_1, cat_image_2]  # 3 images
batch_y = [1, 0, 1]  # 1=cat, 0=dog

outputs = model(batch_x)
# outputs = [[0.8, 0.2],   # 80% cat - correct!
#            [0.3, 0.7],   # 70% dog - correct!
#            [0.6, 0.4]]   # 60% cat - correct but uncertain

# Prediction loss
pred_loss = CrossEntropyLoss(outputs, batch_y)
# pred_loss = 0.223 + 0.357 + 0.511 = 1.091 / 3 = 0.364

# L2 penalty (λ = 0.01)
all_weights = model.parameters()  # 1.23 million weights
weight_sum_sq = sum(w**2 for w in all_weights)  # = 45,200

l2_penalty = (0.01 / 2) * 45,200 = 226

# Total loss
total_loss = 0.364 + 226 = 226.364

# Backward and update
total_loss.backward()  # Computes ∂L/∂w for ALL weights
optimizer.step()       # Updates: w := w - α×(∂L_pred/∂w + λw)

# After update
weight_sum_sq = 44,870  # Decreased! Weight decay at work!
```

---

## 9. Choosing λ (Hyperparameter Tuning)

### **Rule of Thumb:**

| λ value | Effect | When to use |
|---------|--------|-------------|
| **0** | No regularization | Lots of data, simple model |
| **0.0001-0.001** | Very weak | Slight overfitting |
| **0.01** | Moderate | **Default starting point** |
| **0.1** | Strong | Heavy overfitting |
| **1.0+** | Very strong | Extreme overfitting |

### **Grid Search Example:**

```python
lambdas = [0, 0.001, 0.01, 0.1, 1.0]
best_lambda = None
best_val_acc = 0

for lam in lambdas:
    model = train_model(lambda_l2=lam)
    val_acc = evaluate(model, validation_set)
    
    print(f"λ={lam}: Val Acc = {val_acc:.2%}")
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_lambda = lam

# Output:
# λ=0:     Val Acc = 68%  (overfit)
# λ=0.001: Val Acc = 89%
# λ=0.01:  Val Acc = 93%  ← Best!
# λ=0.1:   Val Acc = 86%
# λ=1.0:   Val Acc = 72%  (underfit)

print(f"Best λ: {best_lambda}")
```

---

## 10. Summary: L2 Regularization

### **What L2 Does:**

```
┌─────────────────────────────────────┐
│ L2 Regularization (Ridge / Weight Decay) │
└─────────────────────────────────────┘

ADDS: Penalty term λ/2 × Σw²

EFFECT: Shrinks weights toward zero

GRADIENT: ∂L/∂w = ∂L_pred/∂w + λw

UPDATE: w := w(1-αλ) - α×∂L_pred/∂w
             ↑
        Weight decay!

RESULT: Smaller, more stable weights
        → Better generalization
```

### **Pros and Cons:**

**Pros:**
- ✓ Simple to implement
- ✓ Computationally cheap
- ✓ Works well in practice
- ✓ Smooth, differentiable
- ✓ All weights shrink (distributes info)

**Cons:**
- ✗ Doesn't set weights to exactly zero
- ✗ Adds hyperparameter (λ) to tune
- ✗ Can underfit if λ too large

---

**Next up: L1 Regularization (Lasso) - Similar idea but with a twist!**


# Part 3: L1 Regularization (Lasso)

## 1. Plain English Explanation

### **The Core Difference from L2**

**L2 (Ridge):** "Keep weights small"
- Penalty = λ/2 × (w₁² + w₂² + w₃² + ...)
- Shrinks all weights toward zero
- Weights get small but rarely exactly zero

**L1 (Lasso):** "Keep weights small AND prefer sparsity"
- Penalty = λ × (|w₁| + |w₂| + |w₃| + ...)
- Shrinks weights toward zero
- Many weights become EXACTLY zero!

```
L2 Result:                    L1 Result:
w = [0.3, 0.5, 0.2, 0.8,     w = [0, 0.7, 0, 1.2,
     0.1, 0.4, 0.6, 0.3]          0, 0, 0.9, 0]
     
All weights small            Half the weights are ZERO!
All features used            Only important features used!
```

### **Why Is This Useful?**

**Feature Selection Automatically!**

```
Cat vs Dog Classifier:
64×64×3 = 12,288 pixel features

With L2: All 12,288 features have small weights
         → Uses all pixels (even noisy ones)

With L1: Maybe only 500 features have non-zero weights
         → Uses only important pixels!
         → "Cat has whiskers" ✓
         → "Random noise in corner" ✗ (weight = 0)
```

### **Real-World Analogy**

**Hiring for a job:**

**L2 approach:** 
```
Give everyone small tasks
- Expert: 20% time
- Good person: 15% time  
- Mediocre: 10% time
- Bad person: 5% time

Everyone works a little!
```

**L1 approach:**
```
Fire the bad performers!
- Expert: 40% time
- Good person: 35% time
- Mediocre: 25% time
- Bad person: 0% (FIRED!)

Only keep the best!
```

---

## 2. The Mathematics

### **L1 Regularized Loss:**

$$L_{L1} = \frac{1}{m}\sum_{i=1}^{m}(y_i - \hat{y}_i)^2 + \lambda\sum_{j=1}^{n}|w_j|$$

**Compare with L2:**

| Aspect | L2 | L1 |
|--------|----|----|
| **Formula** | $\lambda/2 \times \sum w_j^2$ | $\lambda \times \sum \|w_j\|$ |
| **Penalty** | Squared weights | Absolute values |
| **Gradient** | $\lambda w$ (proportional to weight) | $\lambda \times \text{sign}(w)$ (constant!) |
| **Effect** | Weights → small | Weights → zero |
| **Sparsity** | No | Yes |

**Key difference in gradient:**

```
L2 gradient: ∂L/∂w = ∂L_pred/∂w + λw
             If w=10  → adds +10λ
             If w=1   → adds +1λ
             If w=0.1 → adds +0.1λ
             (Gentle push, proportional to size)

L1 gradient: ∂L/∂w = ∂L_pred/∂w + λ×sign(w)
             If w=10  → adds +λ
             If w=1   → adds +λ  
             If w=0.1 → adds +λ
             (Constant push, same for any positive w!)
```

---

## 3. Why L1 Creates Sparsity (Intuition)

### **The Gradient Behavior**

```
L2 Gradient vs Weight:        L1 Gradient vs Weight:

 ∂L/∂w                         ∂L/∂w
   ↑                             ↑
10 │        ╱                  1 │────────────────
 5 │      ╱                    0 │
 0 │────●─────→ w              -1│────────────────
-5 │  ╱                          └────────────────→ w
-10│╱                             
   
Gradient grows with w         Gradient is constant!
Large w → large push          Any w → same push
Small w → tiny push           Keeps pushing until w=0!
```

**What this means:**

**L2:**
```
w = 10  → Push of -10λ → w becomes 9.9
w = 1   → Push of -1λ  → w becomes 0.99
w = 0.1 → Push of -0.1λ → w becomes 0.099
w = 0.01 → Push of -0.01λ → w becomes 0.0099
...
Never quite reaches zero!
```

**L1:**
```
w = 10  → Push of -λ → w becomes 9.9
w = 1   → Push of -λ → w becomes 0.9
w = 0.1 → Push of -λ → w becomes 0.0
        (if λ=0.1 and learning rate makes this happen)
        
Can hit zero exactly!
```

---

## 4. Step-by-Step Numerical Example

### **Scenario: Single Neuron (Comparing L1 vs L2)**

```
Input: x = [2.0, 1.5, 3.0]
Weights: w = [w₁, w₂, w₃]
Bias: b = 0.5
True label: y = 1.0

Forward: z = w₁×2.0 + w₂×1.5 + w₃×3.0 + 0.5
```

---

### **Initial State:**

```
w₁ = 2.0
w₂ = 0.3  (small weight)
w₃ = 5.0
b = 0.5
```

**Forward pass:**
```
z = 2.0×2.0 + 0.3×1.5 + 5.0×3.0 + 0.5
  = 4.0 + 0.45 + 15.0 + 0.5
  = 19.95

ŷ = sigmoid(19.95) ≈ 0.9999999975

Prediction loss:
L_pred = (1.0 - 0.9999999975)² ≈ 0.00000000006
```

---

### **Case 1: L2 Regularization (λ = 0.1)**

**L2 penalty:**
```
L2 = (λ/2) × (w₁² + w₂² + w₃²)
   = (0.1/2) × (2.0² + 0.3² + 5.0²)
   = 0.05 × (4.0 + 0.09 + 25.0)
   = 0.05 × 29.09
   = 1.4545

Total loss:
L = 0.00000000006 + 1.4545 ≈ 1.4545
```

**Gradients:**
```
∂L_pred/∂w₁ ≈ 0 (prediction almost perfect)
∂L_pred/∂w₂ ≈ 0
∂L_pred/∂w₃ ≈ 0

L2 gradients:
∂L2/∂w₁ = λw₁ = 0.1 × 2.0 = 0.2
∂L2/∂w₂ = λw₂ = 0.1 × 0.3 = 0.03
∂L2/∂w₃ = λw₃ = 0.1 × 5.0 = 0.5

Total gradients:
∂L/∂w₁ ≈ 0 + 0.2 = 0.2
∂L/∂w₂ ≈ 0 + 0.03 = 0.03
∂L/∂w₃ ≈ 0 + 0.5 = 0.5
```

**Update (learning rate α = 0.1):**
```
w₁ := 2.0 - 0.1×0.2 = 2.0 - 0.02 = 1.98
w₂ := 0.3 - 0.1×0.03 = 0.3 - 0.003 = 0.297
w₃ := 5.0 - 0.1×0.5 = 5.0 - 0.05 = 4.95
```

**After 100 iterations:**
```
w₁ = 1.23
w₂ = 0.19  ← Still non-zero!
w₃ = 3.08

All weights shrunk proportionally
Small weight (w₂) stayed small but non-zero
```

---

### **Case 2: L1 Regularization (λ = 0.1)**

**L1 penalty:**
```
L1 = λ × (|w₁| + |w₂| + |w₃|)
   = 0.1 × (|2.0| + |0.3| + |5.0|)
   = 0.1 × (2.0 + 0.3 + 5.0)
   = 0.1 × 7.3
   = 0.73

Total loss:
L = 0.00000000006 + 0.73 ≈ 0.73
```

**L1 gradients:**
```
∂L1/∂w₁ = λ × sign(w₁) = 0.1 × sign(2.0) = 0.1 × 1 = 0.1
∂L1/∂w₂ = λ × sign(w₂) = 0.1 × sign(0.3) = 0.1 × 1 = 0.1
∂L1/∂w₃ = λ × sign(w₃) = 0.1 × sign(5.0) = 0.1 × 1 = 0.1

Notice: All gradients are 0.1 (same magnitude!)
Doesn't matter if weight is 0.3 or 5.0!
```

**Total gradients:**
```
∂L/∂w₁ ≈ 0 + 0.1 = 0.1
∂L/∂w₂ ≈ 0 + 0.1 = 0.1  ← Same as w₁!
∂L/∂w₃ ≈ 0 + 0.1 = 0.1  ← Same as w₁!
```

**Update (learning rate α = 0.1):**
```
w₁ := 2.0 - 0.1×0.1 = 2.0 - 0.01 = 1.99
w₂ := 0.3 - 0.1×0.1 = 0.3 - 0.01 = 0.29
w₃ := 5.0 - 0.1×0.1 = 5.0 - 0.01 = 4.99

All decrease by same absolute amount (0.01)
Proportionally, w₂ decreased more!
```

**After 30 iterations:**
```
Iteration 1:  w₂ = 0.29
Iteration 10: w₂ = 0.20
Iteration 20: w₂ = 0.10
Iteration 30: w₂ = 0.00  ← Reached zero!

w₂ hits zero at iteration 30!
After that, w₂ stays at 0 (sparse!)

w₁ = 1.70 (still large, still useful)
w₂ = 0.00 ← KILLED!
w₃ = 4.70 (still large, still useful)
```

---

### **Detailed Evolution Over Time:**

| Iteration | L2: w₂ | L1: w₂ | L2: Gradient | L1: Gradient |
|-----------|--------|--------|--------------|--------------|
| 0 | 0.300 | 0.300 | 0.030 | 0.100 |
| 5 | 0.285 | 0.250 | 0.029 | 0.100 |
| 10 | 0.272 | 0.200 | 0.027 | 0.100 |
| 15 | 0.259 | 0.150 | 0.026 | 0.100 |
| 20 | 0.247 | 0.100 | 0.025 | 0.100 |
| 25 | 0.236 | 0.050 | 0.024 | 0.100 |
| 30 | 0.225 | **0.000** | 0.023 | **0.000** |
| 50 | 0.196 | **0.000** | 0.020 | **0.000** |
| 100 | 0.153 | **0.000** | 0.015 | **0.000** |

**Key observations:**

```
L2 Behavior:
- Gradient decreases as weight shrinks
- Takes forever to reach zero
- Weight asymptotically approaches zero
- Never exactly zero

L1 Behavior:
- Gradient constant until weight hits zero
- Reaches zero in finite time!
- Once zero, gradient becomes zero (stays dead)
- Sparse solution!
```

---

## 5. The "Soft Thresholding" Effect

### **What Happens Near Zero:**

**L1 has a special property near zero:**

```
If |∂L_pred/∂w| < λ:
  → L1 penalty dominates
  → Weight gets pushed to exactly zero!
  
Example:
∂L_pred/∂w = 0.05
λ = 0.1

Total gradient = 0.05 + 0.1×sign(w) = 0.15
Weight decreases until it crosses zero,
then the sign flips and it gets pushed back to zero!
Net effect: w = 0 (stuck at zero)
```

**Visualization:**

```
    ∂L/∂w
      ↑
  0.2 │         ╱ L_pred gradient only
      │       ╱
  0.1 │─────●───── λ = 0.1 (L1 penalty line)
      │   ╱   ╲
    0 │─●───────●── w
      │         ╲
 -0.1 │───────────
      
Between the two ●'s: L1 penalty > L_pred gradient
→ Weight gets pushed to zero and stays there!

This zone is called the "sparse region"
```

---

## 6. Complete Example: Cat vs Dog with L1

### **Network Architecture (Same as before):**

```
Input: 12,288 pixels
Hidden: 100 neurons  
Output: 2 neurons

Total weights: 1,229,000
```

### **Training with L1 (λ = 0.001)**

---

#### **Epoch 1:**

**Initial weights (random, small):**
```
Sample of 10 weights:
w = [0.02, -0.01, 0.03, -0.02, 0.01, 
     0.04, -0.03, 0.02, -0.01, 0.03]

All weights non-zero: 1,229,000 / 1,229,000 = 100%
```

**Forward pass (one training image: cat):**
```
Prediction: ŷ = [0.51, 0.49]  (51% cat - barely right)
True label: y = [1, 0]

L_pred = CrossEntropyLoss = 0.673
```

**L1 penalty:**
```
Sum of absolute weights:
Σ|w| = 6,145.3

L1_penalty = λ × Σ|w|
           = 0.001 × 6,145.3
           = 6.145

Total loss:
L = 0.673 + 6.145 = 6.818
```

**L1 gradients (sample):**
```
For w = 0.02:  ∂L1/∂w = 0.001 × sign(0.02) = 0.001
For w = -0.01: ∂L1/∂w = 0.001 × sign(-0.01) = -0.001
For w = 0.03:  ∂L1/∂w = 0.001 × sign(0.03) = 0.001

All have magnitude 0.001!
```

**After update:**
```
Small weights that contribute little:
- If |∂L_pred/∂w| < 0.001: Weight moves toward zero
- Some weights hit zero!

Active weights: 1,227,500 / 1,229,000 = 99.9%
(1,500 weights already zeroed out!)
```

---

#### **Epoch 10:**

```
Training accuracy: 78%
Test accuracy: 76%

Sample weights:
w = [0.00, 0.00, 1.23, -0.00, 0.45,
     2.10, 0.00, 0.87, -0.00, 1.56]
     
Active weights: 982,000 / 1,229,000 = 79.9%
Zero weights: 247,000 (20% are dead!)

L_pred = 0.421
L1_penalty = 0.001 × 3,892.1 = 3.892
Total loss = 4.313
```

**What's happening:**
```
Network is learning:
- Important features (cat's ears, whiskers): Large weights
- Unimportant features (background noise): Zero weights

Feature selection in progress!
```

---

#### **Epoch 50:**

```
Training accuracy: 94%
Test accuracy: 91%

Active weights: 412,000 / 1,229,000 = 33.5%
Zero weights: 817,000 (66.5% are dead!)

Network uses only 1/3 of original features!

Sample weights:
w = [0.00, 0.00, 3.45, 0.00, 0.00,
     5.23, 0.00, 2.87, 0.00, 4.12]

Important pixel features:
- Pixel (23, 34): w = 5.23  (cat ear detector!)
- Pixel (45, 12): w = 4.12  (whisker detector!)
- Pixel (67, 89): w = 0.00  (random background - ignored)

L_pred = 0.095
L1_penalty = 0.001 × 2,145.7 = 2.146
Total loss = 2.241
```

---

#### **Final Model (Epoch 100):**

```
Training accuracy: 96%
Test accuracy: 93%

Active weights: 287,000 / 1,229,000 = 23.4%
Zero weights: 942,000 (76.6% sparse!)

Network uses only 287K parameters instead of 1.23M!
- 76.6% reduction in model size!
- Faster inference!
- Better interpretability!

Weight distribution:
┌─────────────────────────────────┐
│ Weight Value  │  Count          │
├─────────────────────────────────┤
│  0.0 (exactly)│  942,000 (76.6%)│ ← Sparsity!
│  0.0 to 1.0   │   98,000        │
│  1.0 to 3.0   │  142,000        │
│  3.0 to 5.0   │   38,000        │
│  5.0+         │    9,000        │
└─────────────────────────────────┘

Most important features:
Top 5 non-zero weights:
1. w[234,567] = 8.9   (strong cat ear signal)
2. w[123,456] = 7.2   (whisker pattern)
3. w[789,012] = 6.8   (dog nose shape)
4. w[345,678] = 6.1   (fur texture)
5. w[901,234] = 5.9   (eye position)
```

---

## 7. Comparing L1 vs L2 on Same Network

### **Same Cat vs Dog Network, Same Data:**

| Metric | L2 (λ=0.01) | L1 (λ=0.001) |
|--------|-------------|--------------|
| **Training Acc** | 94% | 96% |
| **Test Acc** | 92% | 93% |
| **Non-zero weights** | 1,229,000 (100%) | 287,000 (23%) |
| **Zero weights** | 0 (0%) | 942,000 (77%) |
| **Model size** | 4.9 MB | 1.1 MB ↓ |
| **Inference time** | 12 ms | 4 ms ↓ |
| **Weight magnitude** | Avg: 0.8 | Avg: 2.1 (for non-zero) |

---

### **Weight Distribution Comparison:**

```
L2 Distribution:                L1 Distribution:

  Count                          Count
    ↑                              ↑
500K│    ╱╲                    900K│█
    │   ╱  ╲                       │█
300K│  ╱    ╲                      │█
    │ ╱      ╲                     │█
100K│╱        ╲                100K│ ╱╲
    └──────────────→ Weight        └────────────→ Weight
   -2  -1  0  1  2               0   2   4   6

Bell curve around 0            Spike at exactly 0!
All weights used              + Few large weights
                              "Sparse" solution
```

---

### **Feature Importance:**

**L2 Result:**
```
All 12,288 pixels used with small weights:
Pixel 1:    w = 0.03  (ear - important)
Pixel 2:    w = 0.02  (whisker - important)
...
Pixel 5000: w = 0.005 (background noise - useless but included)
...
Pixel 12288: w = 0.001 (corner - useless but included)

Cannot tell which features are important!
All features contribute a little.
```

**L1 Result:**
```
Only 2,156 pixels (17%) have non-zero weights:
Pixel 1:    w = 2.34  (ear - IMPORTANT! ✓)
Pixel 2:    w = 1.89  (whisker - IMPORTANT! ✓)
...
Pixel 5000: w = 0.00  (background noise - IGNORED! ✓)
...
Pixel 12288: w = 0.00  (corner - IGNORED! ✓)

Clear feature selection!
Easy to interpret: "Model uses these 2,156 pixels"
```

---

## 8. Geometric Intuition: Why L1 Creates Sparsity

### **Constraint Regions:**

**L2 Constraint: $w_1^2 + w_2^2 \leq C$**

```
    w₂
     ↑
   1 │   ●─────●
     │ ●         ●
   0 ├●──────────●─→ w₁
     │ ●         ●
  -1 │   ●─────●
  
Circle (smooth, round)
Solution can be anywhere on circle
Rarely hits axes (w₁=0 or w₂=0)
```

**L1 Constraint: $|w_1| + |w_2| \leq C$**

```
    w₂
     ↑
   1 │      ●
     │     ╱ ╲
   0 ├────●───●──→ w₁
     │     ╲ ╱
  -1 │      ●
  
Diamond (sharp corners!)
Solution tends to hit corners
Corners are on axes → sparse!
(Either w₁=0 or w₂=0)
```

### **Optimization with Contours:**

```
L2 Case:                       L1 Case:

    w₂                            w₂
     ↑                             ↑
     │  ╱○○○╲                      │  ╱○○○╲
     │ ○     ○ ← Loss contours     │ ○     ○
     │○   ●   ○                    │○   ●   ○
     │ ○ ╱●╲ ○                     │ ○     ○
     │  ●───●                      │  ●─────●
     │    L2                       │     ╱│╲
     │  circle                     │   ╱  │  ╲
     └───────────→ w₁              └─────┼────→ w₁
         ↑                               ●
    Solution:                      Solution hits axis!
    w=[0.3, 0.5]                   w=[0, 0.7] ← Sparse!
    Both non-zero                  w₁ exactly zero!
```

**Why corners matter:**
```
Probability solution hits axis:
- Circle (L2): Low (must hit exact point)
- Diamond (L1): High (corners ARE on axes!)

In high dimensions:
- L2: Almost never hits axis (no sparsity)
- L1: Frequently hits axes (lots of sparsity!)
```

---

## 9. Proximal Gradient Descent (How to Handle L1's Non-Differentiability)

### **The Problem:**

L1 penalty $|w|$ is not differentiable at $w=0$!

```
    |w|
     ↑
   2 │      ╱
   1 │    ╱
   0 │──●──────→ w
    -2  0  2
        ↑
    Not smooth at zero!
    Gradient is undefined!
```

### **The Solution: Soft Thresholding**

Instead of regular gradient descent, use:

$$w^{new} = \text{SoftThreshold}(w^{old} - \alpha \nabla L_{pred}, \alpha\lambda)$$

**Soft Threshold Function:**

$$\text{SoftThreshold}(x, \theta) = \begin{cases}
x - \theta & \text{if } x > \theta \\
0 & \text{if } |x| \leq \theta \\
x + \theta & \text{if } x < -\theta
\end{cases}$$

---

### **Step-by-Step Example:**

```
Current weight: w = 0.5
Learning rate: α = 0.1
L1 strength: λ = 0.01
Prediction gradient: ∂L_pred/∂w = 2.0

Step 1: Regular gradient step
w_temp = w - α × ∂L_pred/∂w
       = 0.5 - 0.1 × 2.0
       = 0.5 - 0.2
       = 0.3

Step 2: Apply soft threshold with αλ = 0.1 × 0.01 = 0.001
w_new = SoftThreshold(0.3, 0.001)
      = 0.3 - 0.001  (since 0.3 > 0.001)
      = 0.299

If w_temp was 0.0005:
w_new = SoftThreshold(0.0005, 0.001)
      = 0  (since |0.0005| < 0.001)
      Weight gets killed!
```

### **Numerical Comparison:**

| w_temp | Threshold (αλ) | w_new | What Happened |
|--------|----------------|-------|---------------|
| 2.0 | 0.01 | 1.99 | Shrunk slightly |
| 0.5 | 0.01 | 0.49 | Shrunk slightly |
| 0.1 | 0.01 | 0.09 | Shrunk slightly |
| 0.02 | 0.01 | 0.01 | Shrunk to near zero |
| 0.005 | 0.01 | **0.00** | Killed! (sparse) |
| -0.008 | 0.01 | **0.00** | Killed! (sparse) |
| -0.5 | 0.01 | -0.49 | Shrunk slightly |

**Visual representation:**

```
Before soft threshold:          After soft threshold:
w = [...0.5, 0.02, 0.005,      w = [...0.49, 0.01, 0.00,
     -0.008, -0.5, 0.1...]          0.00, -0.49, 0.09...]
                                     ↑     ↑
                                Killed!  Killed!
```

---

## 10. Practical Implementation

### **PyTorch-style Code:**

```python
import torch
import torch.nn as nn

class L1Regularization:
    def __init__(self, lambda_l1=0.001):
        self.lambda_l1 = lambda_l1
    
    def __call__(self, model):
        """Compute L1 penalty for all model parameters"""
        l1_penalty = 0
        for param in model.parameters():
            l1_penalty += torch.sum(torch.abs(param))
        return self.lambda_l1 * l1_penalty

# Training loop
model = CatDogClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
l1_reg = L1Regularization(lambda_l1=0.001)

for epoch in range(100):
    for batch_x, batch_y in train_loader:
        # Forward pass
        outputs = model(batch_x)
        
        # Prediction loss
        pred_loss = criterion(outputs, batch_y)
        
        # L1 penalty
        l1_penalty = l1_reg(model)
        
        # Total loss
        total_loss = pred_loss + l1_penalty
        
        # Backward and optimize
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
        # Optional: Hard threshold very small weights to exactly zero
        with torch.no_grad():
            for param in model.parameters():
                param[torch.abs(param) < 1e-6] = 0
```

---

### **Complete Training Example with Logging:**

```python
# Training one epoch with detailed logging
lambda_l1 = 0.001
alpha = 0.1

print("Before training:")
non_zero_before = sum((p != 0).sum().item() 
                     for p in model.parameters())
print(f"  Non-zero weights: {non_zero_before:,}")

for batch_idx, (images, labels) in enumerate(train_loader):
    # Forward
    outputs = model(images)  # Shape: (batch_size, 2)
    # outputs = [[0.7, 0.3],  # 70% cat
    #            [0.2, 0.8],  # 80% dog
    #            [0.6, 0.4]]  # 60% cat
    
    # Prediction loss
    pred_loss = F.cross_entropy(outputs, labels)
    # pred_loss = 0.357
    
    # L1 penalty
    l1_penalty = 0
    for param in model.parameters():
        l1_penalty += torch.abs(param).sum()
    l1_penalty = lambda_l1 * l1_penalty
    # l1_penalty = 0.001 × 1,128,945 = 1,128.945
    
    # Total loss
    total_loss = pred_loss + l1_penalty
    # total_loss = 0.357 + 1,128.945 = 1,129.302
    
    # Backward
    optimizer.zero_grad()
    total_loss.backward()
    
    # Soft thresholding (built into the gradient)
    # PyTorch automatically handles this through the L1 gradient
    
    # Update
    optimizer.step()
    
    # Hard thresholding (optional, for true sparsity)
    with torch.no_grad():
        for param in model.parameters():
            # Kill weights smaller than threshold
            mask = torch.abs(param) < 1e-4
            param[mask] = 0
    
    if batch_idx % 10 == 0:
        # Count non-zero weights
        non_zero = sum((p != 0).sum().item() 
                      for p in model.parameters())
        sparsity = (1 - non_zero / non_zero_before) * 100
        
        print(f"Batch {batch_idx}:")
        print(f"  Loss: {total_loss.item():.4f}")
        print(f"  Non-zero weights: {non_zero:,}")
        print(f"  Sparsity: {sparsity:.1f}%")

print("\nAfter training:")
non_zero_after = sum((p != 0).sum().item() 
                    for p in model.parameters())
sparsity_final = (1 - non_zero_after / non_zero_before) * 100
print(f"  Non-zero weights: {non_zero_after:,}")
print(f"  Final sparsity: {sparsity_final:.1f}%")
```

**Output:**
```
Before training:
  Non-zero weights: 1,229,000

Batch 0:
  Loss: 1,129.3024
  Non-zero weights: 1,227,834
  Sparsity: 0.1%

Batch 10:
  Loss: 845.2134
  Non-zero weights: 1,198,456
  Sparsity: 2.5%

Batch 20:
  Loss: 623.4521
  Non-zero weights: 1,145,789
  Sparsity: 6.8%

...

Batch 100:
  Loss: 234.5678
  Non-zero weights: 892,341
  Sparsity: 27.4%

After training:
  Non-zero weights: 287,234
  Final sparsity: 76.6%
```

---

## 11. When to Use L1 vs L2

### **Decision Guide:**

| Scenario | Best Choice | Reason |
|----------|-------------|--------|
| **High-dimensional data** (many features) | L1 | Feature selection needed |
| **Most features relevant** | L2 | Keep all features |
| **Want interpretability** | L1 | See which features matter |
| **Want faster inference** | L1 | Fewer non-zero weights |
| **Want smaller model** | L1 | Sparsity reduces size |
| **Features correlated** | L2 | L1 picks one randomly |
| **Stable gradient flow** | L2 | Smooth everywhere |
| **Don't care about size** | L2 | Easier to optimize |

---

### **Practical Examples:**

**Use L1 for:**

```
1. Medical diagnosis (1000s of gene expressions)
   → Want to know: "Which 20 genes matter?"
   → L1 gives sparse solution: 980 weights = 0

2. Text classification (100,000 vocabulary)
   → Want to know: "Which 500 words are most indicative?"
   → L1 selects important words, zeros out rare ones

3. Mobile deployment
   → Need small model (limited memory/compute)
   → L1 gives 70% sparsity → 3× smaller model

4. Image features (12,288 pixels)
   → Want to know: "Which pixels detect cats?"
   → L1 shows: "These 2,000 pixels around ears/whiskers"
```

**Use L2 for:**

```
1. Image classification (already few features after conv layers)
   → All features important
   → L2 keeps all, makes smooth

2. General-purpose model
   → No specific feature selection needed
   → L2 easier to train, more stable

3. Small networks
   → Model size not a concern
   → L2 gives slight performance edge

4. Highly correlated features
   → L1 would pick one arbitrarily
   → L2 distributes weights fairly across correlates
```

---

### **Elastic Net (Combining L1 and L2):**

Sometimes best to use BOTH!

$$L_{elastic} = L_{pred} + \lambda_1 \sum|w_i| + \frac{\lambda_2}{2}\sum w_i^2$$

```
Gets benefits of both:
- L1: Sparsity, feature selection
- L2: Stability, handles correlated features

Example:
λ₁ = 0.001 (L1 for sparsity)
λ₂ = 0.01 (L2 for stability)

Result:
- 60% sparsity (from L1)
- Stable training (from L2)
- Best of both worlds!
```

---

## 12. Summary: L1 Regularization

### **What L1 Does:**

```
┌─────────────────────────────────┐
│   L1 Regularization (Lasso)     │
└─────────────────────────────────┘

ADDS: Penalty term λ × Σ|w|

EFFECT: Shrinks weights to EXACTLY zero

GRADIENT: ∂L/∂w = ∂L_pred/∂w + λ×sign(w)
                                ↑
                          Constant magnitude!

UPDATE: w := w - α×∂L_pred/∂w - α×λ×sign(w)

RESULT: Sparse weights (many exactly zero)
        → Feature selection
        → Smaller models
        → Better interpretability
```

---

### **L1 vs L2 Comparison:**

| Aspect | L2 (Ridge) | L1 (Lasso) |
|--------|------------|------------|
| **Penalty** | $\frac{\lambda}{2}\sum w^2$ | $\lambda \sum \|w\|$ |
| **Gradient** | $\lambda w$ (proportional) | $\lambda \cdot \text{sign}(w)$ (constant) |
| **Sparsity** | No | Yes ✓ |
| **Feature Selection** | No | Yes ✓ |
| **Differentiability** | Smooth | Non-smooth at 0 |
| **Model Size** | Full | Reduced ✓ |
| **Interpretability** | Hard | Easy ✓ |
| **Training Speed** | Faster | Slightly slower |
| **Correlated Features** | Keeps all | Picks one |

---

### **Key Takeaways:**

1. **L1 creates sparsity** because constant gradient pushes small weights to exactly zero

2. **Geometric interpretation:** Diamond-shaped constraint with corners on axes

3. **Feature selection:** Automatically identifies important features

4. **Practical benefit:** Smaller models, faster inference, better interpretability

5. **Trade-off:** Slightly harder to optimize (non-smooth), but worth it for sparsity

---

**Next: Dropout - A completely different approach to regularization!** 
